In [12]:
import cryptocompare
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')

In [13]:
cryptocompare_API_key = 'd06d4760932b2b4a3f581de5a75d7e8e84ab84257a9880105c9ac47f4afa3fca'
cryptocompare.cryptocompare._set_api_key_parameter(cryptocompare_API_key)
print("API Key set!")

API Key set!


In [14]:
# Add EMA28, MACD

def drop_columns(df, colnames=['volumefrom', 'volumeto']):
    for cn in colnames:
        try:
          df.drop(cn, inplace=True, axis=1)
        except:
            pass
    return df

def add_ema(df, ema=28):
    # df['backward_ewm'] = df['Close'].ewm(span=ema,min_periods=0,adjust=False,ignore_na=False).mean()
    df = df.sort_index()
    df['ewm'] = df['close'].ewm(span=ema,min_periods=0,adjust=False,ignore_na=False).mean()
    return df

def add_macd(df, e1=12, e2=26, e3=9):
    df = df.sort_index()
    exp1 = df['close'].ewm(span=e1, adjust=False).mean()
    exp2 = df['close'].ewm(span=e2, adjust=False).mean()
    macd = exp1 - exp2
    macd_signal = macd.ewm(span=e3, adjust=False).mean()
    df['macd_histo'] = macd - macd_signal
    return df

def add_returns(df):
    df['return'] = df['close'].pct_change(1)
    return df


def prepare(df):
    df = add_returns(add_macd(add_ema(drop_columns(df))))
    return df

In [15]:
# Parameters
ticker_symbol = 'BTC'
currency = 'USD'
limit_value_hour = 2000 
limit_value_minute = 2000
exchange_name = 'CCCAGG'
bull_market_hourly_timestamp_end = datetime(2023, 2, 1, 0, 0)
bear_market_hourly_timestamp_end = datetime(2022, 2, 1, 0, 0)

bull_market_minute_timestamp_end = datetime(2023, 2, 16, 0, 0)
crab_market_minute_timestamp_end = datetime(2023, 2, 18, 0, 0)

In [16]:
# Hourly data: bear
raw_price_data_bear_hour = \
    cryptocompare.get_historical_price_hour(
        ticker_symbol,
        currency,
        limit=limit_value_hour,
        exchange=exchange_name,
        toTs=bear_market_hourly_timestamp_end
    )

# Convert the raw price data into a DataFrame
hourly_price_data_bear = pd.DataFrame.from_dict(raw_price_data_bear_hour)

# Set the time columns as index and convert it to datetime
hourly_price_data_bear.set_index("time", inplace=True)
hourly_price_data_bear.index = pd.to_datetime(hourly_price_data_bear.index, unit='s')
hourly_price_data_bear['datetimes'] = hourly_price_data_bear.index
hourly_price_data_bear['datetimes'] = hourly_price_data_bear['datetimes'].dt.strftime(
    '%Y-%m-%d')

hourly_data_bear = prepare(hourly_price_data_bear.iloc[:, :6])
hourly_data_bear.to_pickle('./hourly_bear.pkl')

In [17]:
# Hourly data: bull
raw_price_data_bull_hour = \
    cryptocompare.get_historical_price_hour(
        ticker_symbol,
        currency,
        limit=limit_value_hour,
        exchange=exchange_name,
        toTs=bull_market_hourly_timestamp_end
    )

# Convert the raw price data into a DataFrame
hourly_price_data_bull = pd.DataFrame.from_dict(raw_price_data_bull_hour)

# Set the time columns as index and convert it to datetime
hourly_price_data_bull.set_index("time", inplace=True)
hourly_price_data_bull.index = pd.to_datetime(hourly_price_data_bull.index, unit='s')
hourly_price_data_bull['datetimes'] = hourly_price_data_bull.index
hourly_price_data_bull['datetimes'] = hourly_price_data_bull['datetimes'].dt.strftime(
    '%Y-%m-%d')

hourly_data_bull = prepare(hourly_price_data_bull.iloc[:, :6])
hourly_data_bull.to_pickle('./hourly_bull.pkl')

In [18]:
# Minute data: bull
raw_price_data_bull_minute = \
  cryptocompare.get_historical_price_minute(
    ticker_symbol,
    currency,
    limit=limit_value_minute,
    exchange=exchange_name,
    toTs=bull_market_minute_timestamp_end
  )

# Convert the raw price data into a DataFrame
minutely_price_data_bull = pd.DataFrame.from_dict(raw_price_data_bull_minute)

# Set the time columns as index and convert it to datetime
minutely_price_data_bull.set_index("time", inplace=True)
minutely_price_data_bull.index = pd.to_datetime(minutely_price_data_bull.index, unit='s')
minutely_price_data_bull['datetimes'] = minutely_price_data_bull.index
minutely_price_data_bull['datetimes'] = minutely_price_data_bull['datetimes'].dt.strftime(
  '%Y-%m-%d')

minutely_data_bull = prepare(minutely_price_data_bull.iloc[:, :6])
minutely_data_bull.to_pickle('./minutely_bull.pkl')

In [19]:
# Minute data: crab
raw_price_data_bear_minute = \
    cryptocompare.get_historical_price_minute(
        ticker_symbol,
        currency,
        limit=limit_value_minute,
        exchange=exchange_name,
        toTs=crab_market_minute_timestamp_end
    )

# Convert the raw price data into a DataFrame
minutely_price_data_bear = pd.DataFrame.from_dict(raw_price_data_bear_minute)

# Set the time columns as index and convert it to datetime
minutely_price_data_bear.set_index("time", inplace=True)
minutely_price_data_bear.index = pd.to_datetime(minutely_price_data_bear.index, unit='s')
minutely_price_data_bear['datetimes'] = minutely_price_data_bear.index
minutely_price_data_bear['datetimes'] = minutely_price_data_bear['datetimes'].dt.strftime(
    '%Y-%m-%d')

minutely_data_crab = prepare(minutely_price_data_bear.iloc[:, :6])
minutely_data_crab.to_pickle('./minutely_crab.pkl')